In [24]:
!pip install ome-zarr

Load libraries, define some functions

In [25]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nbvv
from upath import UPath as Path
from aicsimageio import AICSImage
from ome_zarr.reader import Reader
from ome_zarr.io import parse_url
import logging
logging.getLogger("bfio").setLevel(logging.ERROR)
logging.getLogger("aicsimageio").setLevel(logging.ERROR)


def read_ome_zarr(path, level=0, image_name="default"):
    path = str(path if image_name is None else Path(path) / image_name)
    reader = Reader(parse_url(path))

    node = next(iter(reader()))
    pps = node.metadata["coordinateTransformations"][0][0]["scale"][-3:]
   
    return AICSImage(
        node.data[level].compute(),
        channel_names=node.metadata["name"],
        physical_pixel_sizes=pps
    )

def rescale_image(img_data, channels):
    img_data = img_data.squeeze().astype(np.float32)
    
    for ix, channel in enumerate(channels):
        if "_seg" not in channel:
            img_data[ix] -= 1
            
            img_data[ix] = np.where(
                img_data[ix] >= 0,
                img_data[ix] / img_data.max(),
                -1
            )
    return img_data.astype(np.float16)

Load the dataframe

In [26]:
cells_df = pd.read_parquet("s3://variance-dataset/processed/manifest.parquet")
print(f'Number of cells: {len(cells_df)}')
print(f'Number of columns: {len(cells_df.columns)}')

Number of cells: 215081
Number of columns: 1242


In [27]:
some_cell = cells_df.sample(1).iloc[0]

In [28]:
full_img = read_ome_zarr(some_cell["registered_path"])

In [29]:
full_img.channel_names

['bf',
 'dna',
 'membrane',
 'structure',
 'dna_segmentation',
 'membrane_segmentation',
 'struct_segmentation_roof']

In [30]:
img_data = full_img.data.squeeze()

In [31]:
img_data.shape

(7, 136, 245, 381)

Now let's examine this data in a 3d viewer.  `nbvv` is a multichannel viewer for interactively examining volume data.

We display our image with the `volshow` command.   

`volshow` accepts either a 3d or 4d data array.  If the array is 4D, it will be treated as multi-channel. 

You can also tell `volshow` the physical spacing between the volume pixels in X, Y and Z dimensions (`spacing`).  In our case the data has already been re-scaled to have equal-sized x, y, and z pixels so we just pass in \[1,1,1\] (the defaults) as an example.

We also provide the names of the channels (`channel_names`) for display.

Here are some of the main features you will use in the user interface:

3D / XY / XZ / YZ
Click these buttons to switch between full 3D view and side projections.  

In the side projection modes, you will see one cross-sectional "slice" through the volume at a time.  

You can change the sections you are viewing using the "Clipping" tab at the bottom of the viewport.

For multi-channel (4D CZYX) data, you can switch displayed channels on and off by clicking the `volume` checkbox next to each channel.
You can also change the colors for each channel by clicking a swatch to the left.
Note that the left panel is scrolling, and if you have many channels you may need to scroll.
You can do contrast adjustment using the Settings (gear wheel) button for each channel.  



In [36]:
nbvv.volshow(
    img_data,
    spacing=[1,1,1],  # full_img.physical_pixel_sizes,
    channel_names=full_img.channel_names
)

VolumeWidget(dimensions={'tile_width': 204, 'tile_height': 136, 'rows': 15, 'cols': 10, 'atlas_width': 2040, '…

The viewer has automatically tried to apply some contrast adjustment, but we could easily pre-apply contrast adjustment by doing math on the data array before passing it to the viewer.  

The viewer will accept any scalar-valued 3d or 4d array we give it.

In [33]:
img_data_rescaled = rescale_image(img_data, full_img.channel_names)

In [34]:
img_data_rescaled.shape

(7, 136, 245, 381)

In [35]:
nbvv.volshow(
    img_data_rescaled,
    spacing=[1,1,1],#full_img.physical_pixel_sizes,
    channel_names=full_img.channel_names
)

VolumeWidget(dimensions={'tile_width': 204, 'tile_height': 136, 'rows': 15, 'cols': 10, 'atlas_width': 2040, '…